### Webscraping Google Scholar - User Profile

#### Reference: 
Medium Article I found about scraping google scholar [here](https://proxiesapi-com.medium.com/scraping-google-scholar-with-python-and-beautifulsoup-850cbdfedbcf)

Stackoverflow starter code [here](https://stackoverflow.com/questions/67146312/web-scraping-google-scholar-author-profiles)

Reference Medium Article from SerpeAPI [here](https://python.plainenglish.io/scrape-google-scholar-with-python-fc6898419305#daf8)

In general, use **selenium** if you need to interact with the webpage

You can also use scholarly package (mine didn't work because I keep getting fake user agent error)

Pagination solution: using 'cstart' and 'pagesize' to work with Show More option

Down the line: incoroporating object oriented programming - author could be a class



### Where I am now:
Next steps:
1. 

In [2]:
from bs4 import BeautifulSoup as bs
import requests, lxml, os, re, pandas as pd

In [54]:
def getting_author_info(user_id):
  headers = {
      'User-agent':
      "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
  }

  proxies = {
    'http': os.getenv('HTTP_PROXY')
  }
  params={
    'user':user_id,
    'hl':'en', 
    'cstart':0, 
    'pagesize':'100'
    }
  
  html = requests.get('https://scholar.google.com/citations', headers=headers, proxies=proxies, params=params).text
  soup = bs(html, 'lxml')
  profile = soup.find("div", id="gsc_prf_w")
  user_info = dict()
  user_info["scholar_id"] = user_id
  user_info["scholar_name"] = profile.find("div", id="gsc_prf_in").string
  # getting string for both institution and po
  pos_ins = str(profile.find("div", {"class":"gsc_prf_il"}).get_text()).rsplit(", ", 1)
  if len(pos_ins) > 1:
    user_info["position"], user_info["affiliated_institution"] = str(profile.find("div", {"class":"gsc_prf_il"}).get_text()).rsplit(", ", 1)
  else: 
    user_info["position"] = "No Position Available"
    user_info["affiliated_institution"] = pos_ins[0]
  tags = profile.find_all("a", {"class": "gsc_prf_inta gs_ibl"})
  user_info["tags"] = [tag.string for tag in tags]
  return user_info
  
  

### Hurdle for Getting CoAuthors
1. Write getting co_authors function
    - currently not able to find the co-authors ul tag .... bc the testing profile did not have co-authors section....
2. was only able to scrape the top 20 co-authors before we have to click "View All" and maybe use selenium
    - found alternative way by scraping a new view option offered by google scholar. Reference [here](https://datascience-enthusiast.com/R/google_scholar_R.html)

In [90]:
def getting_coauthors(user_id):
    headers = {
      'User-agent':
      "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
    }

    proxies = {
        'http': os.getenv('HTTP_PROXY')
    }
    params={
        'user':user_id,
        'hl':'en'
        }

    
    
    html = requests.get('https://scholar.google.com/citations?view_op=list_colleagues', headers=headers, proxies=proxies, params=params).text
   
    soup = bs(html, 'lxml')
    co_authors = soup.find_all("div", {"class": "gs_ai_t gs_ai_pss"}) 
    all_ca_names, all_ca_user_ids, all_ca_positions, all_ca_institutions = ([] for _ in range(4))
    for co_author in co_authors:

        all_ca_names += [co_author.find("a").string]
        all_ca_user_ids += [re.search(r'(?<=user=)[^&#]*', co_author.find("a")["href"]).group(0)]
        pos_ins = str(co_author.find("div", {"class": "gs_ai_aff"}).get_text()).rsplit(", ", 1)
        if len(pos_ins) > 1:
            all_ca_positions += [pos_ins[0]]
            all_ca_institutions += [pos_ins[1]]
        else:
            all_ca_positions += ["No Position Available"]
            all_ca_institutions += [pos_ins[0]]        
                


    df = pd.DataFrame({
            "user_id": user_id,
            "co_author_name": all_ca_names,
            "co_author_ids": all_ca_user_ids,
            "co_author_positions": all_ca_positions,
            "co_author_affiliated_institutions": all_ca_institutions
        })
    return df

In [31]:

def getting_authors_articles(user_id):
  headers = {
      'User-agent':
      "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
  }

  proxies = {
    'http': os.getenv('HTTP_PROXY')
  }
  params={
    'user':user_id,
    'hl':'en', 
    'cstart':0, 
    'pagesize':'100'
    }
  
  all_titles, all_urls, all_authors, all_years, all_journals, all_cite_times = ( [] for _ in range(6))

  while True:
    html = requests.get('https://scholar.google.com/citations', headers=headers, proxies=proxies, params=params).text

    soup = bs(html, 'lxml')
    table = soup.find("tbody", id="gsc_a_b")
    rows = table.find_all("tr", {"class": 'gsc_a_tr'})
    for row in rows:
      
      a_tag = row.find("a", {"class": "gsc_a_at"})
      ## extract paper titles
      all_titles += [a_tag.string]
      
      ## extract paper url
      if "href" in a_tag.attrs:
        all_urls += ["https://scholar.google.com"+a_tag["href"]]
      else: all_urls += ["No Link Available"]
      ## extract author names
      authors = row.select("td > a + div")
      if len(authors) > 0:
        all_authors += [author.string for author in authors]
      else: all_authors += ["No Author Available"]
      ## extract published year
      years = row.select("td > a + div + div span")
      if len(years) > 0:
        all_years += [str(year.string).replace(", ", "") for year in years]
      else: all_years += ["No Year Available"]
      
      ## extract journal name
      journals = row.select("td > a + div + div")
      
      ## extract citation times
      freq = row.find("a", {"class": "gsc_a_ac gs_ibl"}).string
      if freq == None:
        all_cite_times += ["No Citation Available"]
      else:
        all_cite_times += [row.find("a", {"class": "gsc_a_ac gs_ibl"}).string]
      
      ## check if there are multiple pages, if so, go to the next page
      if len(journals) > 0:
        all_journals += [re.sub(r', [0-9]{4}$', '', journal.get_text()) if journal.get_text() != ''
                         else 'No Journal Available'
                         for journal in journals]
      else: all_journals += ["No Journal Available"]
      
    if len(rows)<int(params['pagesize']):
      break
    else:
      params['cstart'] += 100
    
  df = pd.DataFrame({"scholar_id":user_id,"paper_titles": all_titles, "author_names": all_authors, 
                     "journals": all_journals, "published_years":all_years,
                     "citation_times": all_cite_times, "paper_urls": all_urls})
  
  return df

df = getting_authors_articles("acmtRMAAAAAJ")




## Testing Code

In [88]:
user_id = '2M9ZhTcAAAAJ'
headers = {
      'User-agent':
      "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
  }

proxies = {
    'http': os.getenv('HTTP_PROXY')
  }
params={
    'user':user_id,
    'hl':'en'
    }
html = requests.get('https://scholar.google.com/citations?view_op=list_colleagues', headers=headers, proxies=proxies, params=params).text

soup = bs(html, 'lxml')


#     ## extract paper names
# names = table.find_all("a", {"class": "gsc_a_at"})
# names[0]["href"]
#names = [name.string for name in names]
co_authors = soup.find_all("div", {"class": "gs_ai_t gs_ai_pss"})
href = co_authors[0].find("h3").find("a")["href"]
re.search(r'(?<=user=)[^&#]*', href).group(0)
#re.search(r'(?<=user=).*[^A-Za-z0-9]+', "user=_90dfjkalv&").group(0)
# for co_author in co_authors:
#   co_author.find("a")

#soup


'acmtRMAAAAAJ'